In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.cuda
import torch.optim as optim
import torchvision.transforms.functional as TF
import torchvision.transforms as transforms
import os
from PIL import Image
from tqdm.notebook import tqdm
from torch.nn.functional import relu
from torch.utils.data import DataLoader, Dataset
from SHG import SHG
from utils import *

In [9]:
""" PATHS """
MODEL_PATH = "C:/Users/André/OneDrive 2/OneDrive/Skrivebord/bsc_data/models/Mon_Apr_12_18-11-59_2021/epoch_46.pth"
AVERAGE_RGB_PATH = "C:/Users/André/OneDrive 2/OneDrive/Skrivebord/bachelor_thesis/code/average_rgb.npy"

TRAIN_INPUT_PATH = "C:/Users/André/OneDrive 2/OneDrive/Skrivebord/bsc_data/train/image/"

VAL_INPUT_PATH = "C:/Users/André/OneDrive 2/OneDrive/Skrivebord/bsc_data/validation/image/"
VAL_OUTPUT_PATH = "C:/Users/André/OneDrive 2/OneDrive/Skrivebord/bsc_data/validation/heatmaps/"

SAVING_PATH_VAL = "C:/Users/André/OneDrive 2/OneDrive/Skrivebord/bsc_data/latent_space_data/val_full_skeleton/"
SAVING_PATH_TRAIN_ALL = "C:/Users/André/OneDrive 2/OneDrive/Skrivebord/bsc_data/latent_space_data/train_all/"
SAVING_PATH_VAL_ALL = "C:/Users/André/OneDrive 2/OneDrive/Skrivebord/bsc_data/latent_space_data/val_all/"

In [3]:
""" LOADING """
average_rgb = np.loadtxt(AVERAGE_RGB_PATH)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SHG(num_hourglasses=1).to(device)
model.load_state_dict(torch.load(MODEL_PATH))

<All keys matched successfully>

In [11]:
""" Dataset and dataloader """
class dataset_full(Dataset):
    def __init__(self, X_path, y_path, average_rgb):
        self.X_path = X_path
        self.y_path = y_path
        self.X_data = os.listdir(self.X_path)
        self.average_rgb = average_rgb
        self.norm = transforms.Normalize(mean = self.average_rgb, std = [1, 1, 1])

    def __len__(self):
        return len(self.X_data)

    def __getitem__(self, i):
        ID = self.X_data[i]
        x = Image.open(self.X_path + ID)

        y = []

        for i in range(17):
            y.append(torch.from_numpy(np.load(self.y_path + ID[:-4] + "/" + str(i) + ".npy")))

        x = TF.to_tensor(x)

        if (x.shape[0] == 1): # If the image is gray-scale, cast it to rgb
            x = torch.stack((x[0],) * 3)

        x = self.norm(x) # Subtracts mean rgb

        y = torch.stack(y)
        return x, y, ID

val_dataset = dataset_full(VAL_INPUT_PATH, VAL_OUTPUT_PATH, average_rgb)
val_dataloader = DataLoader(val_dataset)


class dataset_all(Dataset):
    def __init__(self, X_path, average_rgb):
        self.X_path = X_path
        self.X_data = os.listdir(self.X_path)
        self.average_rgb = average_rgb
        self.norm = transforms.Normalize(mean = self.average_rgb, std = [1, 1, 1])

    def __len__(self):
        return len(self.X_data)

    def __getitem__(self, i):
        ID = self.X_data[i]
        x = Image.open(self.X_path + ID)

        x = TF.to_tensor(x)

        if (x.shape[0] == 1): # If the image is gray-scale, cast it to rgb
            x = torch.stack((x[0],) * 3)

        x = self.norm(x) # Subtracts mean rgb

        return x, ID

train_dataset = dataset_all(TRAIN_INPUT_PATH, average_rgb)
train_dataloader = DataLoader(train_dataset)

val_dataset_all = dataset_all(VAL_INPUT_PATH, average_rgb)
val_dataloader_all = DataLoader(val_dataset_all)

In [ ]:
""" Creating validation data of full skeletons """
model.eval()
with torch.no_grad():
    for x, y_heatmaps, ID in tqdm(val_dataloader, total = len(val_dataloader)):
        ID = ID[0][:-4]
        x = x.to(device, dtype = torch.float)
        y_kp = turn_featuremaps_to_keypoints(y_heatmaps)
        y_kp = np.array(y_kp).reshape((-1, 3))

        if (0 not in y_kp[:, -1]):
            _, bottleneck = model(x)
            bottleneck = bottleneck[-1][0]
            bottleneck = bottleneck.cpu().data.numpy()
            np.save(SAVING_PATH_VAL + ID + ".npy", bottleneck)

In [7]:
""" Creating train data of all skeletons """
model.eval()
with torch.no_grad():
    for x, ID in tqdm(train_dataloader, total = len(train_dataloader)):
        ID = ID[0][:-4]
        x = x.to(device, dtype = torch.float)

        _, bottleneck = model(x)
        bottleneck = bottleneck[-1][0]
        bottleneck = bottleneck.cpu().data.numpy()
        np.save(SAVING_PATH_TRAIN_ALL + ID + ".npy", bottleneck)

In [12]:
""" Creating val data of all skeletons """
model.eval()
with torch.no_grad():
    for x, ID in tqdm(val_dataloader_all, total = len(val_dataloader_all)):
        ID = ID[0][:-4]
        x = x.to(device, dtype = torch.float)

        _, bottleneck = model(x)
        bottleneck = bottleneck[-1][0]
        bottleneck = bottleneck.cpu().data.numpy()
        np.save(SAVING_PATH_VAL_ALL + ID + ".npy", bottleneck)